In [1]:
import openai
from qdrant_client import QdrantClient

## Embedding function

In [6]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

## Retrieval function

In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [28]:
def retrieve_data(query: str, qdrant_client: QdrantClient , k:int =5):

    query_embedding= get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [ ]:
retrieved_context = retrieve_data("What kind of earphones can I get?", qdrant_client)

In [12]:
retrieved_context

{'retrieved_context_ids': ['B0CBMPG524',
  'B0B9FTVL58',
  'B0C142QS8X',
  'B0C6K1GQCF',
  'B0BNHVLF7G'],
 'retrieved_context': ['Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music. 【Hear Your Surroundings】Open earbuds that rest on your ears without co

### Format retrieved context

In [10]:
def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context

In [13]:
preprocessed_context = process_context(retrieved_context)

In [14]:
print(preprocessed_context)

- ID: B0CBMPG524, rating: 4.4, description: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music. 【Hear Your Surroundings】Open earbuds that rest on your ears without covering them, you can hear your music and your surroundings at the same time. Whether y

### Create prompt

In [ ]:
def build_prompt(preprocessed_context: str, question: str) -> str:

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [21]:
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")

In [22]:
print(prompt)


You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- ID: B0CBMPG524, rating: 4.4, description: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headp

In [23]:
def generate_answer(prompt:str):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content

In [24]:
print(generate_answer(prompt))

You can choose from several types in stock, including:
- Open-ear wireless earbuds with earhooks (B0CBMPG524)
- Wireless/in-ear Bluetooth earbuds with microphone (B0B9FTVL58)
- Kids over-ear headphones (B0C142QS8X)
- Wireless earbuds with noise-cancelling mic (B0C6K1GQCF)
- Open-ear bone conduction headphones (Siniffo X15, B0BNHVLF7G)

If you tell me what features you want (open-ear, waterproof, battery life, kids use, etc.), I can Recommend a couple of options.


# RAG pipeline

In [26]:
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer

In [29]:
print(rag_pipeline("What kind of earphones can I get fro my son?"))

You can consider kid-friendly options like:
- TUNEAKE Kids Headphones: over-ear, volume-limited to 94dB, wired with 3.5mm jack. Quiet, foldable, adjustable, and comfortable for kids.
- QearFun Cat Earbuds for Kids: wired in-ear earbuds with a microphone and a cute case (Blue), designed for kids.

If you’re open to wireless:
- TELSOR Wireless Earbuds: Bluetooth earbuds with noise-cancelling mic, long battery life, IPX7 waterproof.

Also, Open Ear Headphones are available if you prefer a design that doesn’t go inside the ear:
- Open Ear Headphones: Bluetooth 5.3 with long playback, IPX7, open-ear design for awareness.

Would you like recommendations narrowed to wired for school use or wireless for travel?
